In [1]:
import pandas as pd
import numpy as np

import os
# import tmdbsimple as tmdb

In [2]:
os.makedirs('Data/', exist_ok=True)
os.listdir('Data/')

['ratings.csv.gz', 'title_aka.csv.gz', 'title_basics.csv.gz']

In [3]:
# basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
# basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

basics = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1,"Comedy,Short"


In [4]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9117063 entries, 0 to 9117062
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   tconst          object 
 1   titleType       object 
 2   primaryTitle    object 
 3   originalTitle   object 
 4   isAdult         float64
 5   startYear       float64
 6   endYear         float64
 7   runtimeMinutes  object 
 8   genres          object 
dtypes: float64(3), object(6)
memory usage: 626.0+ MB


In [5]:
basics = basics.replace({'\\N': np.nan})
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1,"Comedy,Short"


In [6]:
basics.value_counts('isAdult')

isAdult
0.0       8833772
1.0        283281
2019.0          3
2020.0          2
1981.0          1
2005.0          1
2014.0          1
2017.0          1
dtype: int64

In [7]:
basics[['isAdult', 'startYear',
        'endYear']] = basics[['isAdult', 'startYear',
                              'endYear']].apply(pd.to_numeric)
basics.dtypes

tconst             object
titleType          object
primaryTitle       object
originalTitle      object
isAdult           float64
startYear         float64
endYear           float64
runtimeMinutes     object
genres             object
dtype: object

In [8]:
# title_aka_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'
# title_aka = pd.read_csv(title_aka_url,sep='\t', low_memory=False)

title_aka = pd.read_csv("Data/title_aka.csv.gz", low_memory=False)
title_aka.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [9]:
# ratings_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz'
# ratings = pd.read_csv(ratings_url,sep='\t', low_memory=False)

ratings = pd.read_csv("Data/ratings.csv.gz", low_memory=False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1901
1,tt0000002,5.9,254
2,tt0000003,6.5,1695
3,tt0000004,5.7,166
4,tt0000005,6.2,2513


In [10]:
# # Save current dataframe to file.
# basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
# ratings.to_csv("Data/ratings.csv.gz",compression='gzip',index=False)
# title_aka.to_csv("Data/title_aka.csv.gz",compression='gzip',index=False)

In [11]:
basics.dropna(subset=['runtimeMinutes', 'genres'], inplace=True)

In [12]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1,"Comedy,Short"


In [13]:
basics = basics[basics['titleType'].str.contains('movie') == True]

In [14]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary', case=False)
basics = basics[~is_documentary]

In [15]:
basics = basics[basics["startYear"] >= 2000]

In [16]:
title_aka = title_aka.replace({'\\N': np.nan})
title_aka.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [17]:
title_aka = title_aka[title_aka['region'] == 'US']
title_aka.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [18]:
keepers = basics['tconst'].isin(title_aka['titleId'])
keepers

34790       True
61086       True
67631       True
77925      False
86762       True
           ...  
9116735     True
9116744     True
9116783    False
9116828     True
9116912    False
Name: tconst, Length: 141831, dtype: bool

In [19]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34790,tt0035423,movie,Kate & Leopold,Kate & Leopold,0.0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61086,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0.0,2020.0,NaN,70,Drama
67631,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0.0,2018.0,NaN,122,Drama
86762,tt0088751,movie,The Naked Monster,The Naked Monster,0.0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
91031,tt0093119,movie,Grizzly II: Revenge,Grizzly II: The Predator,0.0,2020.0,NaN,74,"Horror,Music,Thriller"
...,...,...,...,...,...,...,...,...,...
9116199,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0.0,2019.0,NaN,74,Drama
9116595,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0.0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
9116735,tt9916170,movie,The Rehearsal,O Ensaio,0.0,2019.0,NaN,51,Drama
9116744,tt9916190,movie,Safeguard,Safeguard,0.0,2020.0,NaN,90,"Action,Adventure,Thriller"


In [20]:
title_aka.region.value_counts()

US    1339046
Name: region, dtype: int64

In [21]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82097 entries, 34790 to 9116828
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          82097 non-null  object 
 1   titleType       82097 non-null  object 
 2   primaryTitle    82097 non-null  object 
 3   originalTitle   82097 non-null  object 
 4   isAdult         82097 non-null  float64
 5   startYear       82097 non-null  float64
 6   endYear         0 non-null      float64
 7   runtimeMinutes  82097 non-null  object 
 8   genres          82097 non-null  object 
dtypes: float64(3), object(6)
memory usage: 6.3+ MB


In [22]:
title_aka.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1339046 entries, 5 to 32726003
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1339046 non-null  object
 1   ordering         1339046 non-null  int64 
 2   title            1339046 non-null  object
 3   region           1339046 non-null  object
 4   language         3653 non-null     object
 5   types            963040 non-null   object
 6   attributes       44568 non-null    object
 7   isOriginalTitle  1337671 non-null  object
dtypes: int64(1), object(7)
memory usage: 91.9+ MB


In [23]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1257048 entries, 0 to 1257047
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1257048 non-null  object 
 1   averageRating  1257048 non-null  float64
 2   numVotes       1257048 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.8+ MB


# TMDB

In [24]:
import json, time
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [25]:
with open(r"C:\Users\nbeac\.secret\tmdb_api.json", 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [26]:
tmdb.API_KEY = login['api-key']

In [27]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['ratings.csv.gz', 'title_aka.csv.gz', 'title_basics.csv.gz']

In [28]:
## make a movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)

In [29]:
## movie objects have a .info dictionary 
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/n2nm4aZRmXyJ9LT4xQX9X6ThcP7.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 79.309,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 174,
   'logo_path': '/IuAlhI9eVC9Z8UQWOIDdWRKSEJ.png'

In [30]:
print(info['budget'])
print(info['revenue'])
print(info['imdb_id'])

movie = tmdb.Movies('tt1361336')
info = movie.info()
print(info['budget'])

63000000
463517383
tt0133093
50000000


In [31]:
# example from package README
response = movie.releases()
for c in movie.countries:
    if c['iso_3166_1'] == 'US':
        print(c['certification'])

PG
PG
PG


In [32]:
# Get the movie object for the current id
movie = tmdb.Movies('tt1361336')
# save the .info .releases dictionaries
info = movie.info()
releases = movie.releases()
# Loop through countries in releases
for c in releases['countries']:
    # if the country abbreviation==US
    if c['iso_3166_1' ] =='US':
        ## save a "certification" key in the info dict with the certification
       info['certification'] = c['certification']


In [33]:
def get_movie_with_rating(movie_id):
    ## Get movie and release dates
    movie = tmdb.Movies(movie_id)
    ## Construct output dict
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            movie_info['certification'] = c['certification']
    return [movie_info]
test = get_movie_with_rating("tt0848228") #put your function name here
test

[{'adult': False,
  'backdrop_path': '/nNmJRkg8wWnRmzQDe2FwKbPIsJV.jpg',
  'belongs_to_collection': {'id': 86311,
   'name': 'The Avengers Collection',
   'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
   'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
  'budget': 220000000,
  'genres': [{'id': 878, 'name': 'Science Fiction'},
   {'id': 28, 'name': 'Action'},
   {'id': 12, 'name': 'Adventure'}],
  'homepage': 'https://www.marvel.com/movies/the-avengers',
  'id': 24428,
  'imdb_id': 'tt0848228',
  'original_language': 'en',
  'original_title': 'The Avengers',
  'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
  'popularity': 295.288,
  'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
  'production_companies': [{'id':

# Before the Loops

In [34]:
YEARS_TO_GET = [2000, 2001]

# OUTER Loop

## Setting up a Progress Bar

Select a JSON_FILE filename to save the results in progress.
* Check if the file exists.
    * if no:
        * Create the empy JSON_FILE with "with open" that just contains the key "imdb_id"
            
    * if yes:
        * Do nothing!

The code below will identify the folder in the FOLDER we just define above and will name the file based on the current year

In [36]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    JSON_FILE = f"{FOLDER}tmdb_api_results_{YEAR}.json"
    # Check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    if file_exists == False:
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id':0}],f)    

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

## Define/filter the IDs to call


In [39]:
basics = pd.read_csv(r"C:\Users\nbeac\OneDrive\Documents\GitHub\Dojo-Project-03\Data\title_basics.csv.gz")

#Saving new year as the current df
df = basics.loc[ basics['startYear']==YEAR].copy()
# saving movie ids to list
movie_ids = df['tconst'].copy()#.to_list()

C:\Users\nbeac\AppData\Local\Temp\ipykernel_26512\1151842385.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  basics = pd.read_csv(r"C:\Users\nbeac\OneDrive\Documents\GitHub\Dojo-Project-03\Data\title_basics.csv.gz")


## Check for and remove any previously downloaded Movie id's


In [40]:
# Load existing data from json into a dataframe called "previous_df"
previous_df = pd.read_json(JSON_FILE)

In [41]:
# filter out any ids that are already in the JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

# INNER Loop


In [42]:
def get_movie_with_rating(movie_id):
    ## Get movie and release dates
    movie = tmdb.Movies(movie_id)
    ## Construct output dict
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            movie_info['certification'] = c['certification']
    return [movie_info]

def write_json(new_data, filename):
    with open(filename,'r+') as file:
        file_data = json.load(file)
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)
        
        file.seek(0)
        json.dump(file_data,file)

## Iterate through the list of Movie IDs and make the calls

The below relies on the the "get_movie_with_rating" function that we wrote above. 

In [47]:
# INNER Loop
for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = get_movie_with_rating(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)

        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            continue


    
final_year_df = pd.read_json(JSON_FILE)
    
csv_fname = f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz"
final_year_df.to_csv(csv_fname, compression="gzip", index=False)

Movies from 2001:   0%|          | 0/102575 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
tmdb_results_2000 = pd.read_csv(r"C:\Users\nbeac\OneDrive\Documents\GitHub\Dojo-Project-03\Data\final_tmdb_data_2000.csv.gz")
tmdb_results_2000.head()

In [ ]:
tmdb_results_2001 = pd.read_csv(r"C:\Users\nbeac\OneDrive\Documents\GitHub\Dojo-Project-03\Data\final_tmdb_data_2001.csv.gz")
tmdb_results_2001.head()